In [1]:
import os
import sys

sys.path.insert(0, '..')

import pyarrow.parquet as pq

from pysalient import io as io
from pysalient import visualisation as vis
from pysalient.evaluation import evaluation

In [2]:
sample_data_path = os.path.join("data", "anonymised_sample.parquet")
# count rows
table = pq.read_table(sample_data_path)
print(f"Number of rows: {table.num_rows}")
# print column names
print(table.column_names)
# Convert the 'true_label' column to a pandas Series
true_label_series = table["true_label"].to_pandas()

# Count the number of true labels (1)
true_count = (true_label_series == 1).sum()

print(f"Number of true labels (1): {true_count}")

# Convert the table to a pandas DataFrame for easier grouping
df = table.to_pandas()

# show table
print(df.head(5))


grouped = df.groupby("encounter_id")

# Count the number of unique groups (encounters)
num_groups = df["encounter_id"].nunique()
print(f"Number of unique encounter groups: {num_groups}")

# Calculate the sum of 'true_label' for each group
group_sums = grouped["true_label"].sum()

# Count how many groups have at least one true positive (sum > 0)
groups_with_positives = (group_sums > 0).sum()
print(
    f"Number of encounter groups with at least one true positive: {groups_with_positives}"
)

Number of rows: 21356
['encounter_id', 'event_timestamp', 'culture_event', 'suspected_infection', 'true_label', 'prediction_proba_1', 'prediction_proba_2']
Number of true labels (1): 789
  encounter_id  event_timestamp  culture_event  suspected_infection  \
0     0666505c              2.0            1.0                  1.0   
1     0666505c              2.0            1.0                  1.0   
2     0666505c              3.0            1.0                  1.0   
3     0666505c              3.0            1.0                  1.0   
4     0666505c              4.0            1.0                  1.0   

   true_label  prediction_proba_1  prediction_proba_2  
0           1            0.185334            0.531925  
1           1            0.185334            0.531925  
2           1            0.134316            0.360046  
3           1            0.134316            0.360046  
4           1            0.118005            0.167655  
Number of unique encounter groups: 100
Number of e

In [3]:
# Define the path relative to the project root
# Assuming the notebook is run from the project root or examples/ directory
sample_data_path = os.path.join("data", "anonymised_sample.parquet")

assigned_table_events = None

if os.path.exists(sample_data_path):
    # Use the actual column names identified during inspection directly
    # Ensure these names actually exist based on the printout above!
    assigned_table_events = io.load_evaluation_data(
        source=sample_data_path,
        y_proba_col="prediction_proba_1",
        y_label_col="true_label",
        aggregation_cols="encounter_id",
        timeseries_col="event_timestamp",
        perform_aggregation=False,

        # We don't provide task_col or model_col from the source
        # assign_task_name="AKI",  # Assign this name to the new 'task' column
        # assign_model_name="LogRegress",  # Assign this name to the new 'model' column
    )

    print("\nSuccessfully loaded data with assigned names (Example 1):")
    print(assigned_table_events.schema)
    print(f"\nNumber of rows: {assigned_table_events.num_rows}")

    # Display first few rows to verify new columns
    print("\nFirst 5 rows (with added 'task' and 'model' columns):")
    print(assigned_table_events.slice(0, 5).to_pandas())

else:
    print(
        f"Skipping data loading (Example 1) as file was not found: {sample_data_path}"
    )



Successfully loaded data with assigned names (Example 1):
encounter_id: string
event_timestamp: double
culture_event: double
suspected_infection: double
true_label: int64
prediction_proba_1: float
prediction_proba_2: float
-- schema metadata --
pysalient.io.y_proba_col: 'prediction_proba_1'
pysalient.io.y_label_col: 'true_label'
pysalient.io.timeseries_col: 'event_timestamp'
pysalient.io.aggregation_cols: '["encounter_id"]'

Number of rows: 21356

First 5 rows (with added 'task' and 'model' columns):
  encounter_id  event_timestamp  culture_event  suspected_infection  \
0     0666505c              2.0            1.0                  1.0   
1     0666505c              2.0            1.0                  1.0   
2     0666505c              3.0            1.0                  1.0   
3     0666505c              3.0            1.0                  1.0   
4     0666505c              4.0            1.0                  1.0   

   true_label  prediction_proba_1  prediction_proba_2  
0         

In [4]:
# Define evaluation parameters
eval_modelid = "LogRegress_01"  # Use a generic ID as model wasn't assigned here
eval_filter = "ExampleFilterDummy"  # Describe the data subset
eval_thresholds = (0.1, 0.9, 0.1)  # Range: 0.1, 0.2, ..., 0.9
# eval_thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9] # Example: List of thresholds

# Run the evaluation
evaluation_results = evaluation(
    data=assigned_table_events,  # Use the table loaded with col_map
    modelid=eval_modelid,
    filter_desc=eval_filter,
    thresholds=eval_thresholds,
    decimal_places=3,  # Control rounding of output floats # check that -1 is no rounding.
    calculate_au_ci=True,  # Enable AU CI calculation (uses bootstrap)
    calculate_threshold_ci=True,
    threshold_ci_method="bootstrap",  # Method for threshold CIs (ignored if calculate_threshold_ci=False)
    ci_alpha=0.05,  # 95% CI
    bootstrap_seed=42,  # For reproducible CIs
    bootstrap_rounds=1000,  # Fewer rounds for notebook speed
    force_threshold_zero=True,
    verbosity=1,
    time_to_event_cols={'bc': 'culture_event'},
    aggregation_func='median',
    time_to_event_fillna=0,
    time_unit='hour'
    
)

/mnt/e/dev/pysalient/.venv/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
Metric func '_calculate_npv_boot': All bootstrap rounds failed calculation; cannot compute CI. This may indicate an issue with the metric calculation or the bootstrap sample characteristics.
Metric func '_calculate_ppv_boot': All bootstrap rounds failed calculation; cannot compute CI. This may indicate an issue with the metric calculation or the bootstrap sample characteristics.
Metric func '_calculate_ppv_boot': All bootstrap rounds failed calculation; cannot compute CI. This may indicate an issue with the metric calculation or the bootstrap sample characteristics.
Metric func '_calculate_ppv_boot': All bootstrap rounds failed calculation; cannot compute CI. This may indicate an issue with the metric calculation or the bootstrap sample characteristics.
Metric func '_calculate_pp

In [5]:
# Visualisation
styled_results = vis.format_evaluation_table(
    evaluation_results, decimal_places=3, ci_column=False
)
display(styled_results)

,modelid,filter_desc,threshold,time_to_first_alert_value,time_to_first_alert_unit,AUROC,AUPRC,Prevalence,Sample_Size,Label_Count,TP,TN,FP,FN,PPV,Sensitivity,Specificity,NPV,Accuracy,F1_Score,median_hrs_from_first_alert_to_bc,count_first_alerts_before_bc,count_first_alerts_after_or_at_bc,median_hrs_first_alerts_after_or_at_bc
0,LogRegress_01,ExampleFilterDummy,0.000,2.000000,hour,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,789,0,20567,0,0.037 [0.034 - 0.039],1.000 [1.000 - 1.000],0.000 [0.000 - 0.000],0.000,0.037 [0.034 - 0.039],0.071 [0.067 - 0.076],0.000000,0,16,-1.000000
1,LogRegress_01,ExampleFilterDummy,0.100,2.000000,hour,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,348,17957,2610,441,0.118 [0.106 - 0.129],0.441 [0.406 - 0.476],0.873 [0.869 - 0.878],0.976 [0.974 - 0.978],0.857 [0.852 - 0.862],0.186 [0.169 - 0.203],0.000000,0,28,-2.000000
2,LogRegress_01,ExampleFilterDummy,0.200,7.000000,hour,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,70,20416,151,719,0.317 [0.254 - 0.377],0.089 [0.069 - 0.108],0.993 [0.991 - 0.994],0.966 [0.964 - 0.968],0.959 [0.957 - 0.962],0.139 [0.109 - 0.168],0.000000,0,12,-1.000000
3,LogRegress_01,ExampleFilterDummy,0.300,34.000000,hour,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20565,2,789,0.000 [0.000 - 0.000],0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.965],0.000 [0.000 - 0.000],0.000000,0,0,0.000000
4,LogRegress_01,ExampleFilterDummy,0.400,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000],0.000000,0,0,0.000000
5,LogRegress_01,ExampleFilterDummy,0.500,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000],0.000000,0,0,0.000000
6,LogRegress_01,ExampleFilterDummy,0.600,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000],0.000000,0,0,0.000000
7,LogRegress_01,ExampleFilterDummy,0.700,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000],0.000000,0,0,0.000000
8,LogRegress_01,ExampleFilterDummy,0.800,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000],0.000000,0,0,0.000000
9,LogRegress_01,ExampleFilterDummy,0.900,nan,None,0.706 [0.682 - 0.727],0.120 [0.104 - 0.139],0.037,21356,789,0,20567,0,789,0.000,0.000 [0.000 - 0.000],1.000 [1.000 - 1.000],0.963 [0.961 - 0.966],0.963 [0.961 - 0.966],0.000 [0.000 - 0.000],0.000000,0,0,0.000000
